# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons **BASED ON THE LATEST API 2.3 and above**
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [1]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
# from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib

import pandas as pd

In [34]:
## If making change to a module and need to re-load it 
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\pozzo_admin\\Desktop\\research\\packages\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

### Step 1: Set up the experiment dictionary.
The first step to planning an experiment is to load the experiment variables and inputs from a csv file. **Within the Testing Plans Folder you should find a templete of a testing plan (open in Excel for easier viewing).**

Currently the info is split up between two sections: Variables for creating sample spaces and variables for OT2 commands. You only need to specify for the sections you will use.The experiment dictionary consist of keys being the variable name and the value being the variables value. Keep in mind the order of listed variables assumptions should be the index of one listed variable refers to the same index of a related variable.

Besides the plan, if planning on creating a sample space you will need to:
- Update/verify the csv chemical database with/for component information.
- Provide the path of the updated chemical csv for plan. 

In [3]:
path = r"./Testing Plans/07_12_CdSe_HT_new_protocol.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples.get_experiment_plan(path, chem_path)
plan

C:\Users\pozzo_admin\Desktop\research\packages\OT2-DOE\PlanPrepareProcess\Plan\CreateSamples.py:28: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  plan_dict['Chemical Database'] = chem_data.T.to_dict()


{'Sample Amount': 1.5,
 'Sample Unit': 'mL',
 'Component Shorthand Names': ['OleicAc', 'Oleylamine', 'CdAc', 'Se'],
 'Component Concentration Units': ['molarity',
  'molarity',
  'molarity',
  'molarity'],
 'Component Concentration Linspaces [min, max, n]': [[0, 0.5, 3],
  [0, 0.5, 3],
  [0, 0.1, 3],
  [0, 0.1, 3]],
 'Stock Names': ['OleicAc-stock',
  'Oleylamine-stock',
  'CdAc-TOP-stock',
  'Se-TOP-stock'],
 'Stock Concentration Units': ['volf', 'volf', 'molarity', 'molarity'],
 'Stock Concentrations': [1, 1, 0.5, 1],
 'OT2 Destination Labwares': ['abgene_96_wellplate_2200ul'],
 'OT2 Destination Labware Slots': [2],
 'OT2 Stock Labwares': ['20mlscintillation_12_wellplate_18000ul'],
 'OT2 Stock Labware Slots': [1],
 'OT2 Left Pipette': 'p300_single_gen2',
 'OT2 Left Pipette Aspiration Rate (uL/sec)': 50,
 'OT2 Left Pipette Dispense Rate (uL/sec)': 300,
 'OT2 Left Tipracks': ['opentrons_96_tiprack_300ul'],
 'OT2 Left Tiprack Slots': [10],
 'OT2 Right Pipette': 'p1000_single',
 'OT2 Rig

### Step 2: Select and Create Sampling Space
The sampling method will depend on the type of sample units, the actual called method (i.e. lattic, non-clustering random..etc) and whether you are providing arguments from the plan in step 1 or importing sample information from elsewhere. 

***Note:*** All information will be stored as a dataframe
 
You can utilize the built in functions in functions to create sample concentration spaces as follows:
- concentration_from_csv: Import csv with headers being component name and concentration unit.
- concentration_from_excel: Import from excel with headers being component name and concentration unit.
- concentration_from_linspace: Pull from plan the linspaces to use lattice or random sampling. Optional: Unity filter allows for last indexed component to be used a completing component. 
- concentration_from_list_samplewise: 
- concentration_from_list_componentwise:
 
**The ONE requirement if you plan to use any of the module functions is you adhere to strict naming convention. This allows the functions to identify key values to use to calculate and determine specific informaiton (i.e. mass, volume to pipette). **

The following conventions are:
- component concentration columns: Should have the term "concentration" and the concentration unit name (i.e. DPPC Concentration Molarity). 
-

In [4]:
concentration_df = CreateSamples.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'], unity_filter=False)
# concentration_df

### Step 3: Create Stock Dictionary
The stock dictionary will hold the stock information which will be used to calculate for stock volumes for the OT2.
The stock dictionary function will take 3 main sets of information (all formatted as lists, matched by index). 
- Stock Names: Stock names are to be formatted as solute1-solute2...soluten-solvent-stock i.e. NaCl-water-stock.
    - The solutes and solvents are expected to be in the chemical dictionary provided in step 1 as information such as density and molecular weight will be used in volume calculations.
- Stock Concentration Units: Any conc units supported (see function).
- Stock Concentrations: Value of stock conc.

In [5]:
stock_dict = CreateSamples.stock_dictionary(plan['Stock Names'], 
                                                    plan['Stock Concentration Units'], 
                                                    plan['Stock Concentrations'])
# stock_dict

### Step 3: Calculate Component Amounts Mass And Volumes
The end goal in this component planning is to calculate for stock volumes, this requires knowledge on the component mass/volumes.

For now there exist one general function *determine_component_amounts* which based on the total amount (either volume or mass) and component conc units will use the appropiate function to calculate for component mass and volume amounts. 

Since this step uses information from the chemical database (density and mw), if that information is missing or not avaible then the resulting values will return as nan. To remove these and replace with zero in cases where the component mass/volume is negligible use the argument within the function to specify a fill amount of nan_fill_value = 0.

In [6]:
complete_amounts_added = CreateSamples.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
# complete_amounts_added

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units


### Step 4: Calculate Volumes of Stocks
From the mass/volume of individual chemical components we calculauted along with the concentration and identities of the provided stocks, we are now able to combine those two pieces of information and obtain the volume of stock needed. 

The general function to use Create.calculate_stock_volumes_from_component_concs will autoamatically use the component information either volume or stock depending on the stock which contains the component and its stock unit. The stock unit essentially determines the pathway of whether component mass or volume will be utilized.See ______ for a map of how specific component concentration untis and stock units pair together to calculate for info. 

Note: This will calculate stock volumes not taking into account any interdepdencies such as common solvents or missing volume to complete a desired sample volume. The next step will have a few cases for which specfic functions can calculate and adjust stock volumes. 

In [7]:
stock_volumes_prelim = CreateSamples.calculate_stock_volumes_from_component_concs(plan, complete_amounts_added, stock_dict)
# stock_volumes_prelim

### Step 4a: Adjust/Add Stock Volumes

Case 1: Common solvents, in this case solvents are shared between more than one stock the stock needs to be taken into account if provided as component in the first place. For example if ethanol is present in 2 stocks and ethanol must be a specfic wtf then we must take into account the ethanol added when adding those two stocks. The case may arise where you do not specify the concentration of the common solvent and in that case we do not care about the final concentration of common solvent so no need to track for it. The function you would use is **Create.calculate_common_solvent_residual_volumes()**

Case 2: Filling case, in this case after calculating each stock volume for a sample it may need to be diluted/have the total sample volume completed by a component. The function to use is **Create.complete_missing_volume_with_commmon_solvent()** which you can specify the name of a new stock along with the total completing volume and/or if not provided will automatically look for a single solvent stock and use that to fill.

Case X: Other?

***NOTE: ALWAYS CONVERT THE FINAL mL TO uL AS OT2 NATIVELY READS ALL NUMBERS AS uL USE THE CreateSamples.convert_mL_to_uL() FUNCTION ***

In [8]:
stock_volumes_completed = CreateSamples.complete_missing_volume_with_commmon_solvent(plan['Sample Amount'],stock_volumes_prelim, stock_dict, 'Octadecene')
complete_df = CreateSamples.convert_mL_to_uL(stock_volumes_completed)
# complete_df

In [9]:
new_columns= ['OleicAc concentration molarity', 'Oleylamine concentration molarity',
       'CdAc concentration molarity', 'Se concentration molarity',
       'OleicAc amount mass g', 'OleicAc amount volume uL',
       'Oleylamine amount mass g', 'Oleylamine amount volume uL',
       'CdAc amount mass g', 'CdAc amount volume uL', 'Se amount mass g',
       'Se amount volume uL', 'Octadecene-stock volume uL','OleicAc-stock amount volume uL',
       'Oleylamine-stock amount volume uL', 'CdAc-TOP-stock amount volume uL',
       'Se-TOP-stock amount volume uL' ]

In [10]:
complete_df= complete_df.reindex(columns=new_columns)

In [11]:
complete_df.head(5)

,OleicAc concentration molarity,Oleylamine concentration molarity,CdAc concentration molarity,Se concentration molarity,OleicAc amount mass g,OleicAc amount volume uL,Oleylamine amount mass g,Oleylamine amount volume uL,CdAc amount mass g,CdAc amount volume uL,Se amount mass g,Se amount volume uL,Octadecene-stock volume uL,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL
0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1500.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.000000,0.000000,0.005922,1.231185,1425.0,0.0,0.0,0.0,75.0
2,0.0,0.0,0.00,0.10,0.0,0.0,0.0,0.0,0.000000,0.000000,0.011844,2.462370,1350.0,0.0,0.0,0.0,150.0
3,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.017288,7.387821,0.000000,0.000000,1350.0,0.0,0.0,150.0,0.0
4,0.0,0.0,0.05,0.05,0.0,0.0,0.0,0.0,0.017288,7.387821,0.005922,1.231185,1275.0,0.0,0.0,150.0,75.0


### Step 5: Apply Filters or Other Selection Criteria to Stock Volumes

**Filters**: The types of filters can be added easily through dataframe logic. For example, ones in the CreateSamples module:
   * *filter_general_min_pipette_restriction* = Given the lower of the pipettes you will use to synthesize the samples, will remove anything not in range or zero. 
   * *filter_total_volume_restriction* = Restrict total volume, mostly based on labware constraints. Column selection based on "total". 
   * *filter_general_max_restriction* = provide max value and column name 
   
**Other Selection Critera**
   * *.multiplydf(n, df)* = Will multiply a data frame by n number of times. Useful when making one sample repeatadly.

In [31]:
# complete_df_filtered = CreateSamples.filter_general_min_pipette_restriction(complete_df, 20)
# complete_df_filtered = CreateSamples.filter_general_max_restriction(column_name='Octadecene-stock volume uL',df=complete_df_filtered, max_value=1000)
# complete_df_filtered = CreateSamples.filter_total_volume_restriction(complete_df_filtered,16000)
# complete_df_filtered = CreateSamples.filter_general_max_restriction(complete_df_filtered,1000, 'Octadecene-stock volume uL')

# # blanks = complete_df_filtered[complete_df_filtered['ODE-ethanol-stock amount volume uL'] == 0]
# complete_df_filtered.reset_index(inplace=True, drop=True)
# complete_df_filtered = complete_df_filtered.iloc[::2,:]

# complete_df_filtered.reset_index(inplace=True, drop=True)
# complete_df_filtered = pd.concat([complete_df_filtered,remove, remove]).drop_duplicates(keep=False)
# complete_df_filtered

In [32]:
# complete_df_filtered

### Step 6 (Optional): Calculate Stock Prep Information

We can calculate stock information to make it easier to prepare these stock. This process if semi-general, currently supporting a few stock concentration units = mgperml, volf, wtf, molf and molarity. All of these require the basis of volume as that is what is driving how much sample we make for the OT2. 

The logic behind this relies on the notation of stocks = solute1-solute2-....solvent. Using notations allows for systematic handling of solutes and solvent determining what is needed based on information from a excel based Chemical Database.

*Reccomended*: Use at the minimum the default buffering (extra stock being made) of 40 percent to ensure you do not aspirate any air by mistake.

BROKEN FIXING

In [12]:
stock_volumes = CreateSamples.isolate_common_column(complete_df,'stock').sum(axis=0)/1000
stock_volumes

Octadecene-stock volume uL           83.641002
OleicAc-stock amount volume uL        9.640138
Oleylamine-stock amount volume uL     9.993861
CdAc-TOP-stock amount volume uL      12.150000
Se-TOP-stock amount volume uL         6.075000
dtype: float64

### Step 8: Simulate/Execute
The main information needed to create commands for the OT2 is the position of the stocks and the volumes in respect to each well you would like to move the stock to. All volumes in sent to the OT2 will be interpretted as microliters (uL). 

* Pipetting Strategy 1: Add each stock to all wells then move onto next stock. The upside to this is the simplicity of the written protocol. It has a downside of using more tips. It can allow for evaporation or other time sensitve process to occur in the time between stock additions. This is referred to component wise pipetting.

* Pipetting Strategy 2: Adding all stocks to a ONE well then moving on, this is referred to as sample wise pipetting. 

### Step 8a Optional: Load custom labware dictionary (Remote Testing)
* Provide the path to the directory holding all custom labware. This directory should have custom labware .json files you have previously made and tested, read more here: https://support.opentrons.com/en/articles/3136504-creating-custom-labware-definitions
* The reason we provide this is when working on a device that is not connected to the OT2's Jupyter notebook there is no way to natively use custom labware. So we create a dictionary of custom labware so we can later load into our protocol to primarily simualte and test protocols for execution later once connnected to the OT2's notebook.
    * When using custom labware on the OT2's notebook it pulls from a folder labeled "labware", which is something built into the Opentrons hardware. It has not been tested if the custom labware dictionary will superceed this directory of custom labware if used on the OT2.
    * The custom labware folder should contain everything the OT2 labware folder contains: ADD GDRIVE link to customize
    
* **NOTE: Any function using the custom_labware_dict, the argument is optional so can remove and should receive same output IF working on OT2 computer.**

Just noted: transfer_info will fail if transfer and distribute used together, well not fail jsut need to concat

In [28]:
labware_dir_path = r"Custom Labware/"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)

In [29]:
list(custom_labware_dict.keys())

['Custom Labware\\20mlscintillation_12_wellplate_18000ul\\20mlscintillation_12_wellplate_18000ul',
 'Custom Labware\\abgene_96_wellplate_2200ul\\abgene_96_wellplate_2200ul\\abgene_96_wellplate_2200ul',
 'Custom Labware\\agilent_6_reservoir_47000ul\\agilent_6_reservoir_47000ul',
 'Custom Labware\\falcon_48_wellplate_1500ul\\falcon_48_wellplate_1500ul',
 'Custom Labware\\fischer_24_wellplate_7400ul\\fischer_24_wellplate_7400ul\\fischer_24_wellplate_7400ul',
 'Custom Labware\\nanosampler_48_wellplate_1650ul\\nanosampler_48_wellplate_1650ul']

### Step 9: Set up Loaded Dictionary and Directions for Volume Handling
It is important to know how many stock container will be needed for each stock and be able to provide the correct instructions for OT2 commands to follow.

The ranges of stock refers to the ranges of wells one stock will cover. The range is provided in terms of the index of the stock volume dataframe i.e.([lower, upper])). The format is as follows = [[stock1_range1, stock1_range2....], [stock2_range1, stock2_range2....] []...]. 

The range is based on the provided maximum volume for each container, since variabiltiy exist between filling it is **HIGHLY RECCOMENDED** to underestimate the total volume of stock a container can hold so for a 20mL scitillation vials 16000-17000 should be use as the max. Another reason for this is the OT2 will do its best to pick up as much volume as possible but understand liquid in vials can coat the sides not leaving enough depth for the OT2 to grab, so it is **HIGHLY RECCOMENDED** to have at least a **15-20%** volume buffer for each stock

The basis of range is on the maximum volume of the stock container, however currently limitation are: 
 * All stock containers must be the same

In [35]:
# Test with octadecene first, pipette.touch_tip() for stock, not sure for sample would depend on if component or sample wise making
# # then how to do every other grid, just add rows of zeros!
final_df = CreateSamples.isolate_common_column(complete_df,'stock').round(3)

protocol = simulate.get_protocol_api('2.8', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, plan)
# max_source_vol = 16000
stock_position_info = ALH.stock_well_ranges(final_df, loaded_dict['Stock Wells'], volume_buffer_pct=10) 
stock_position_info

C:\Users\pozzo_admin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\pozzo_admin\.opentrons\deck_calibration.json not found. Loading defaults


{'Octadecene-stock volume uL': {'Ranges': [[0, 12],
   [12, 27],
   [27, 42],
   [42, 59],
   [59, 76],
   [76, 81]],
  'Total Volume': 83641.00499999999,
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'OleicAc-stock amount volume uL': {'Ranges': [[0, 81]],
  'Total Volume': 9640.134,
  'Stock Wells': [B3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'Oleylamine-stock amount volume uL': {'Ranges': [[0, 81]],
  'Total Volume': 9993.860999999999,
  'Stock Wells': [B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'CdAc-TOP-stock amount volume uL': {'Ranges': [[0, 81]],
  'Total Volume': 12150.0,
  'Stock Wells': [C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]

In [82]:
final_df

,Octadecene-stock volume uL,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL
0,1500.00,0.000,0.000,0.0,0.0
1,1425.00,0.000,0.000,0.0,75.0
2,1350.00,0.000,0.000,0.0,150.0
3,1350.00,0.000,0.000,150.0,0.0
4,1275.00,0.000,0.000,150.0,75.0
...,...,...,...,...,...
76,790.21,238.028,246.762,150.0,75.0
77,715.21,238.028,246.762,150.0,150.0
78,715.21,238.028,246.762,300.0,0.0
79,640.21,238.028,246.762,300.0,75.0


In [36]:
Octadecene_directions = ALH.create_sample_making_directions(final_df['Octadecene-stock volume uL'], stock_position_info, loaded_dict, start_position=0)

directions = ALH.create_sample_making_directions(final_df.drop('Octadecene-stock volume uL', axis=1), stock_position_info, loaded_dict, start_position=0)

# directions

In [55]:
Octadecene_directions

{0: {'Octadecene-stock volume uL': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Position': A1 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 1500.0}},
 1: {'Octadecene-stock volume uL': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Position': A2 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 1425.0}},
 2: {'Octadecene-stock volume uL': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Position': A3 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 1350.0}},
 3: {'Octadecene-stock volume uL': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Position': A4 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 1350.0}},
 4: {'Octadecene-stock volume uL': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Position': A5 of Abgene 96 

In [37]:
ALH.add_labware_to_dict(loaded_dict, 'Cleaning Wells', ['agilent_6_reservoir_47000ul'], [9])
ALH.cleaning_tip_protocol(loaded_dict, 3)

Enter Well Index 0
Adding A1 of Agilent 6 Reservoir 47000 ÂµL on 9 for washing step
Enter Cleaning Delay (sec) 0
Enter Times Wanting to mix 0
Enter Well Index 0
Adding A1 of Agilent 6 Reservoir 47000 ÂµL on 9 for washing step
Enter Cleaning Delay (sec) 0
Enter Times Wanting to mix 0
Enter Well Index 0
Adding A1 of Agilent 6 Reservoir 47000 ÂµL on 9 for washing step
Enter Cleaning Delay (sec) 0
Enter Times Wanting to mix 0


{'Cleaning 0': {'well': A1 of Agilent 6 Reservoir 47000 ÂµL on 9,
  'delay': 0.0,
  'mix_n': 0},
 'Cleaning 1': {'well': A1 of Agilent 6 Reservoir 47000 ÂµL on 9,
  'delay': 0.0,
  'mix_n': 0},
 'Cleaning 2': {'well': A1 of Agilent 6 Reservoir 47000 ÂµL on 9,
  'delay': 0.0,
  'mix_n': 0}}

In [39]:
## Octadecene
ALH.pipette_volumes_component_wise(protocol, directions, loaded_dict, mix_before=(1, 1200), touch_tip=True )

Picking up tip from A1 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 1500.0 from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 1000.0 ul
Aspirating 1000.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 1000.0 uL into A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 1000.0 uL/sec
Aspirating 750.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Touching tip
Dispensing 750.0 uL into A1 of Abgene 96 Well Plate 2200 ÂµL on 2 at 1000.0 uL/sec
Touching tip
Mixing 1 times with a volume of 1000.0 ul
Aspirating 1000.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 1000.0 uL into A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 1000.0 uL/sec
Aspirating 750.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Touching tip
Dispensing 750.0 uL into A1 of Abgene 96

Touching tip
Dispensing 246.762 uL into F10 of Abgene 96 Well Plate 2200 ÂµL on 2 at 300.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Blowing out at F10 of Abgene 96 Well Plate 2200 ÂµL on 2
Transferring 246.762 from B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to F11 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 300.0 uL into B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 300.0 uL/sec
Aspirating 246.762 uL from B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Touching tip
Dispensing 246.762 uL into F11 of Abgene 96 Well Plate 2200 ÂµL on 2 at 300.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Blowing out at F11 of Abgene 96 Well Plate 2200 ÂµL on 2
Transferring 246.762 from B4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to F12 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times 

NameError: name 'np' is not defined

In [102]:
## Remaining Stocks
ALH.pipette_volumes_component_wise(protocol, directions, loaded_dict, mix_before=(1, 300), mix_after=(3,300), touch_tip=True )

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 119.014 from B3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A10 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from B3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 300.0 uL into B3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 300.0 uL/sec
Aspirating 119.014 uL from B3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Touching tip
Dispensing 119.014 uL into A10 of Abgene 96 Well Plate 2200 ÂµL on 2 at 300.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Blowing out at A10 of Abgene 96 Well Plate 2200 ÂµL on 2
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A1 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 1500.0 from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A1 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 300.0 ul
Aspirati

Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 300.0 uL into A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 1000.0 uL/sec
Aspirating 531.7955 uL from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Touching tip
Dispensing 531.7955 uL into D11 of Abgene 96 Well Plate 2200 ÂµL on 2 at 1000.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Blowing out at D11 of Abgene 96 Well Plate 2200 ÂµL on 2
Transferring 988.591 from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to D12 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 300.0 uL into A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 1000.0 uL/sec
Aspirating 988.591 uL from A4 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Touching tip
D

Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 300.0 uL into C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 300.0 uL/sec
Aspirating 150.0 uL from C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Touching tip
Dispensing 150.0 uL into D5 of Abgene 96 Well Plate 2200 ÂµL on 2 at 300.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Blowing out at D5 of Abgene 96 Well Plate 2200 ÂµL on 2
Transferring 150.0 from C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to D6 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 300.0 uL into C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 300.0 uL/sec
Aspirating 150.0 uL from C1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Touching tip
Dispensing 150.0 uL i

Dispensing 567.112 uL into F4 of Abgene 96 Well Plate 2200 ÂµL on 2 at 1000.0 uL/sec
Touching tip
Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 300.0 uL into B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 1000.0 uL/sec
Aspirating 567.112 uL from B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Touching tip
Dispensing 567.112 uL into F4 of Abgene 96 Well Plate 2200 ÂµL on 2 at 1000.0 uL/sec
Touching tip
Delaying for 0 minutes and 0.0 seconds
Blowing out at F4 of Abgene 96 Well Plate 2200 ÂµL on 2
Transferring 1059.224 from B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to F5 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 300.0 uL into B1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 1000.0 uL/sec
Aspirating 529.6

Dispensing 300.0 uL into A1 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A1 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Blowing out at A1 of Agilent 6 Reservoir 47000 ÂµL on 9
Delaying for 0 minutes and 3.0 seconds
Mixing 3 times with a volume of 300.0 ul
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Blowing out at A2 of Agilent 6 Reservoir 47000 ÂµL on 9
Delaying for 0 minutes and 2.0 seconds
Mixing 5 times with a volume of 300

Blowing out at A2 of Agilent 6 Reservoir 47000 ÂµL on 9
Delaying for 0 minutes and 2.0 seconds
Mixing 5 times with a volume of 300.0 ul
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Blowing out at A2 of Agilent 6 Reservoir 47000 Âµ

Blowing out at A2 of Agilent 6 Reservoir 47000 ÂµL on 9
Delaying for 0 minutes and 2.0 seconds
Mixing 5 times with a volume of 300.0 ul
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Blowing out at A2 of Agilent 6 Reservoir 47000 Âµ

Blowing out at A1 of Agilent 6 Reservoir 47000 ÂµL on 9
Delaying for 0 minutes and 3.0 seconds
Mixing 3 times with a volume of 300.0 ul
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Blowing out at A2 of Agilent 6 Reservoir 47000 ÂµL on 9
Delaying for 0 minutes and 2.0 seconds
Mixing 5 times with a volume of 300.0 ul
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL

Aspirating 300.0 uL from A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 50.0 uL/sec
Dispensing 300.0 uL into A2 of Agilent 6 Reservoir 47000 ÂµL on 9 at 300.0 uL/sec
Blowing out at A2 of Agilent 6 Reservoir 47000 ÂµL on 9
Delaying for 0 minutes and 4.0 seconds
Transferring 75.0 from C2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to F2 of Abgene 96 Well Plate 2200 ÂµL on 2
Mixing 1 times with a volume of 300.0 ul
Aspirating 300.0 uL from C2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 300.0 uL into C2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 300.0 uL/sec
Aspirating 75.0 uL from C2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Touching tip
Dispensing 75.0 uL into F2 of Abgene 96 Well Plate 2200 ÂµL on 2 at 300.0 uL/sec
Mixing 3 times with a volume of 300.0 ul
Aspirating 300.0 uL from F2 of Abgene 96 Well Plate 2200 ÂµL on 2 at 50.0 uL/sec
Dispensing 300.0 uL into F2 of Abgene 96 Well Plate 2200 ÂµL on 2 at 300.0 uL/sec
Aspi

In [ ]:
new_directions = ALH.create_sample_making_directions(final_df, stock_position_info, loaded_dict, start_position=0)

In [81]:
export_path = r"07_20_21_CdSe_Sample_Info_new_protocol"
export_df = CreateSamples.add_final_location(new_directions,complete_df)
export_df

,UID,Labware,Slot,Well,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity,CdAc amount mass g,CdAc amount volume uL,...,OleicAc amount mass g,OleicAc amount volume uL,Oleylamine amount mass g,Oleylamine amount volume uL,CdAc-TOP-stock amount volume uL,Se-TOP-stock amount volume uL,OleicAc-stock amount volume uL,Oleylamine-stock amount volume uL,Octadecene-stock volume uL,Total Volume
0,S2_A1_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A1,0.00,0.00,0.00,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1500.000000,1500.0
1,S2_A2_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A2,0.00,0.00,0.00,0.25,0.000000,0.000000,...,0.000000,0.000000,0.100309,123.380996,0.0,0.0,0.000000,123.380996,1376.619004,1500.0
2,S2_A3_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A3,0.00,0.00,0.00,0.50,0.000000,0.000000,...,0.000000,0.000000,0.200618,246.761993,0.0,0.0,0.000000,246.761993,1253.238007,1500.0
3,S2_A4_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A4,0.00,0.00,0.25,0.00,0.000000,0.000000,...,0.105922,119.014045,0.000000,0.000000,0.0,0.0,119.014045,0.000000,1380.985955,1500.0
4,S2_A5_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A5,0.00,0.00,0.25,0.25,0.000000,0.000000,...,0.105922,119.014045,0.100309,123.380996,0.0,0.0,119.014045,123.380996,1257.604959,1500.0
5,S2_A6_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A6,0.00,0.00,0.25,0.50,0.000000,0.000000,...,0.105922,119.014045,0.200618,246.761993,0.0,0.0,119.014045,246.761993,1134.223962,1500.0
6,S2_A7_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A7,0.00,0.00,0.50,0.00,0.000000,0.000000,...,0.211845,238.028090,0.000000,0.000000,0.0,0.0,238.028090,0.000000,1261.971910,1500.0
7,S2_A8_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A8,0.00,0.00,0.50,0.25,0.000000,0.000000,...,0.211845,238.028090,0.100309,123.380996,0.0,0.0,238.028090,123.380996,1138.590914,1500.0
8,S2_A9_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A9,0.00,0.00,0.50,0.50,0.000000,0.000000,...,0.211845,238.028090,0.200618,246.761993,0.0,0.0,238.028090,246.761993,1015.209917,1500.0
9,S2_A10_07-02-2021,Bbgene 96 Well Plate 2200 µL,2,A10,0.05,0.05,0.00,0.00,0.017288,7.387821,...,0.000000,0.000000,0.000000,0.000000,150.0,75.0,0.000000,0.000000,1275.000000,1500.0


In [82]:
export_df.to_csv(export_path, index=False)